## 5.1 Optimización de Modelos (Global Average Pooling)

En el notebook anterior decidimos optimizar los siguientes modelos:

 - Rank=1, Name=**SCV-linear-1.0**, Score=**0.9934480277920665 (+/- 0.003944167917138149)**
 - Rank=9, Name=**SGDClassifier**, Score=**0.9916728170032577 (+/- 0.004665749258440611)**
 - Rank=11, Name=**RidgeClassifier-0.8**, Score=**0.9912639128068644 (+/- 0.004574692652304791)**
 - Rank=24, Name=**KNeighborsClassifier-4**, Score=**0.9892162234696844 (+/- 0.00614647322899209)**
 
Para optimizar cada modelo, usaremos GridSearchCV, el cual es un método para automáticamente encontrar los mejores parámetros de un modelo, con base en una métrica objetivo que, en nuestro caso, es la exactitud (_accuracy_).

Las iniciales CV significan que usaremos _cross-validation_ o validación cruzada para estar verdaderamente seguros del desempeño del modelo. Como sudió en la fase de Evaluación de Modelos, haremos 10 validaciones cruzadas por modelo.

Como podemos esperar, este proceso tomará algo de tiempo, dado que no sólo entrenaremos muchos modelos, sino que cada variación necesitará de otros 10 modelos, debido al proceso de validación cruzada.

Empecemos definiendo las funciones auxiliares que usaremos para optimizar.

### Datos

Esta función nos retornará los datos relevantes necesatios para llevar a cabo la optimización de los modelos.

In [1]:
import numpy as np

def load_dataset():
    X = np.load('global_average_features.npy')
    y = np.load('labels.npy')
    
    return X, y

### Modelos

Esta función devuelve un dict de dicts que contiene el modelo a ser optimizado, junto con la grilla de parámetros para hacerlo.

In [2]:
from sklearn.linear_model import SGDClassifier, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

def define_models(models=dict()):
    models['SGDClassifier'] = {
        'model': SGDClassifier(),
        'parameters': {
            'loss': ('hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'),
            'penalty': ('none','l2', 'l1'),
            'alpha': (1e-3, 5e-3, 1e-4, 1e-2, 5e-3),
            'fit_intercept': (True, False)
        }
    }
    
    models['KNeighborsClassifier-4'] = {
        'model': KNeighborsClassifier(n_neighbors=4),
        'parameters': {
            'weights': ('uniform', 'distance'),
            'algorithm': ('auto', 'ball_tree', 'kd_tree', 'brute')
        }
    }
    
    models['SVC-linear-1.0'] = {
        'model': SVC(kernel='linear', C=1.0),
        'parameters': {
            'probability': (True, False),
            'decision_function_shape': ('ovo', 'ovr'),
            'shrinking': (True, False)
        }
    }
    
    models['RidgeClassifier-0.8'] = {
        'model': RidgeClassifier(alpha=0.8),
        'parameters': {
            'fit_intercept': (True, False),
            'alpha': (1e-3, 5e-3, 1e-4, 1e-2, 5e-3),
            'solver': ('svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga')
        }
    }
    
    print(f'Defined {len(models)} models.')

    return models

/usr/lib/python3/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


### Pipeline

Como en el notebook pasado, este pipeline es usado para pre-procesar los _features_ que le pasaremos al modelo.

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

def make_pipeline(model):
    steps = [
        ('StandardScaler', StandardScaler()),
        ('MinMaxScaler', MinMaxScaler()),
        ('model', model)
    ]
    
    pipeline = Pipeline(steps=steps)
    
    return pipeline

### Búsqueda de Paráemtros

Aquí definimos un par de funciones auxiliares que llevarán a cabo la búsqueda de parámetros que optimizarán cada modelo. Al final del proceso, se imprimirán los mejores parámetros, junto con el puntaje/exactitud del mejor clasificador.

In [4]:
from sklearn.model_selection import GridSearchCV

def grid_search_model(X, y, model, folds, metric):
    m = model['model'] 
    parameters = model['parameters']
    
    classifier = GridSearchCV(m, parameters, cv=folds, scoring=metric, n_jobs=-1)
    pipeline = make_pipeline(classifier)
    pipeline.fit(X, y)
    
    return pipeline.steps[-1][1]

In [5]:
import warnings

def grid_search_models(X, y, models, folds=10, metric='accuracy'):
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        for model_name, model in models.items():
            m = grid_search_model(X, y, model, folds, metric)

            if m is not None:
                print(f'Best parameters for {model_name}: \n{m.best_params_}')
                print(f'Best model {metric}: {m.best_score_ * 100}%')
            else:
                print(f'{model_name}: error')
            
            print('----\n')

In [6]:
X, y = load_dataset()
models = define_models()
grid_search_models(X, y, models)

Defined 4 models.
Best parameters for SGDClassifier: 
{'alpha': 0.0001, 'fit_intercept': False, 'loss': 'log', 'penalty': 'none'}
Best model accuracy: 99.2901023890785%
----

Best parameters for KNeighborsClassifier-4: 
{'algorithm': 'brute', 'weights': 'distance'}
Best model accuracy: 99.0853242320819%
----

Best parameters for SVC-linear-1.0: 
{'decision_function_shape': 'ovo', 'probability': True, 'shrinking': True}
Best model accuracy: 99.35836177474403%
----

Best parameters for RidgeClassifier-0.8: 
{'alpha': 0.001, 'fit_intercept': True, 'solver': 'lsqr'}
Best model accuracy: 99.0443686006826%
----



## Conclusión

Estos son nuestros resultados:

 - **SVC-linear-1.0** pasa de 99.34480277920665% a 99.35836177474403%, con una mejora total de 0.01355899553738027%.
 - **RidgeClassifier-0.8** pasa de 99.12639128068644% a 99.0443686006826%, con una desmejora total de 0.08202268000383128%.
 - **SGDClassifier** pasa de 99.16728170032577% a 99.2901023890785%, con una mejora total de 0.122820688752725%.
 - **KNeighborsClassifier-4** pasa de 98.92162234696844% a 99.0853242320819%, con una mejora total de 0.1637018851134684%.
 
Observamos que a excepción de RidgeClassifier-0.8, todos los modelos muestran, al menos, una pequeña mejora en su exactitud, lo que significa que la búsqueda de parámetros fue exitosa.

Claramente, el modelo que más se benefició fue KNeighborsClassifier-4, con una mejora total de 0.1637018851134684%. 

El mejor puntaje lo alcanzó SVC-linear-1.0: 99.35836177474403%.

Lamentablemente, RidgeClassifier-0.8 se desempeña peor al final del procedimiento. Quizás jugar con otros parámetros de mejores resultados.

## Ideas

 - Probar otros modelos.
 - Probar más parámetros.
 - Probar reducción de dimensiones.
 - Usar más técnicas de ingeniería de _features_.
 - Usar _stacking_.
 
### Una nota final

No usamos la versión aplanada de los datos porque no cupo en la memoria de mi computador :)